In [1]:
import os

import pandas as pd
from tensorflow.keras.applications import EfficientNetB0
from tensorflow.keras.applications.efficientnet import preprocess_input
from cloudproject import CloudsClassifier

In [2]:
train_dir = "../data/dataset/train"
valid_dir =  "../data/dataset/valid"
img_height = 200
img_width = 200
batch_size = 32

model_name = "ResNet50V2"

In [7]:
classifier = CloudsClassifier(
    train_dir=train_dir,
    valid_dir=valid_dir,
    img_height=img_height,
    img_width=img_width,
    batch_size=batch_size,
)

Création du dataset d'entraînement...
Chargement des données depuis : ../data/dataset/train
Chargement depuis '../data/dataset/train': 1502 images trouvées pour les classes ['altocumulus', 'altostratus', 'cirrocumulus', 'cirrostratus', 'cirrus', 'cumulonimbus', 'cumulus', 'nimbostratus', 'stratocumulus', 'stratus'].

Création du dataset de validation...
Chargement des données depuis : ../data/dataset/valid
Chargement depuis '../data/dataset/valid': 145 images trouvées pour les classes ['altocumulus', 'altostratus', 'cirrocumulus', 'cirrostratus', 'cirrus', 'cumulonimbus', 'cumulus', 'nimbostratus', 'stratocumulus', 'stratus'].

Les datasets sont prêts.


In [8]:
# model_name = "EfficientNetB0"

# classifier.add_model(
#     name=model_name,
#     model_class=EfficientNetB0,
#     preprocess_function=preprocess_input,
# )

In [9]:
classifier.build_model(base_model_name=model_name)

94668760/94668760 ━━━━━━━━━━━━━━━━━━━━ 2s 0us/step
Modèle construit avec ResNet50V2 comme base.


Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer_2 (InputLayer)      │ (None, 200, 200, 3)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ random_flip (RandomFlip)        │ (None, 200, 200, 3)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ random_rotation                 │ (None, 200, 200, 3)    │             0 │
│ (RandomRotation)                │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ true_divide (TrueDivide)        │ (None, 200, 200, 3)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ subtract (Subtract)             │ (None, 200, 200, 3)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ resnet50v2 (Functional)         │ (None, 7, 7, 2048)     │    23,564,800 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ global_average_pooling2d        │ (None, 2048)           │             0 │
│ (GlobalAveragePooling2D)        │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 2048)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 10)             │        20,490 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 23,585,290 (89.97 MB)

 Trainable params: 20,490 (80.04 KB)

 Non-trainable params: 23,564,800 (89.89 MB)

In [12]:
try:
    classifier.load_model(model_name + ".keras")
except:
    classifier.train(epochs=50, fine_tune_epochs=30, model_name=model_name, learning_rate_ft=1e-6)
    classifier.plot_history()

Erreur lors du chargement du modèle : File not found: filepath=../models\ResNet50V2.keras. Please ensure the file is an accessible `.keras` zip file.

--- Phase 1: Entraînement de la tête de classification ---
Epoch 1/50


InvalidArgumentError: Graph execution error:

Detected at node decode_image/DecodeImage defined at (most recent call last):
<stack traces unavailable>
Error in user-defined function passed to ParallelMapDatasetV2:10 transformation with iterator: Iterator::Root::Prefetch::BatchV2::Shuffle::ParallelMapV2: Unknown image file format. One of JPEG, PNG, GIF, BMP required.
	 [[{{node decode_image/DecodeImage}}]]
	 [[IteratorGetNext]] [Op:__inference_multi_step_on_iterator_13247]

In [11]:
img_path = "../data/dataset/test/Ac-N053-33-_jpg.rf.6b6f2ae38015d73a335d80b7e5cb8f5d.jpg"
img_path = "../data/dataset/test/27949466-091d-47c0-b206-5920d2133e41_jpg.rf.4189ec0e892ac4ceb9be2e1eb04f8e3b.jpg"

predicted_class_name, confidence = classifier.predict(img_path)

UnidentifiedImageError: cannot identify image file <_io.BytesIO object at 0x000001F2F3206660>

In [11]:
class_test = pd.read_csv("../data/dataset/test/_classes.csv", sep=",")

good_answer = 0
wrong_answer = 0
for img_path in class_test["filename"]:
    row = class_test[class_test["filename"] == img_path].iloc[0]
    true_class_name = row[row == 1].index[0]
    predicted_class_name, confidence = classifier.predict(os.path.join(f"../data/dataset/test/{img_path}"), show_image=False)

    if true_class_name == predicted_class_name:
        good_answer +=1
    else:
        wrong_answer +=1

print(f"Bonnes réponses : {good_answer} sur {good_answer + wrong_answer}")
print(f"Mauvaises réponses : {wrong_answer} sur {good_answer + wrong_answer}")

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 94ms/step
Prédiction: altocumulus (88.02%)
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 51ms/step
Prédiction: stratus (96.20%)
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 54ms/step
Prédiction: cirrostratus (41.10%)
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 48ms/step
Prédiction: stratocumulus (51.89%)
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 46ms/step
Prédiction: altostratus (54.20%)
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 46ms/step
Prédiction: cirrus (99.98%)
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 49ms/step
Prédiction: cumulonimbus (99.76%)
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 48ms/step
Prédiction: cumulonimbus (99.87%)
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 47ms/step
Prédiction: stratus (69.16%)
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 46ms/step
Prédiction: nimbostratus (99.68%)
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 46ms/step
Prédiction: cumulonimbus (99.18%)
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 46ms/step
Prédiction: nimbostratus (98.77%)
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 47ms/step
Prédiction: nimbostratus (98.94%)
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 46ms/step
Prédiction: cirrostratus (36.61%)
1/1 ━━━━━